<a href="https://colab.research.google.com/github/pedrolustosab/medlineplus/blob/main/medlineplus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando Bibliotecas
import requests
import re
from bs4 import BeautifulSoup

In [2]:
# URL
infectiousdiseases_url = "https://medlineplus.gov/infections.html"

In [3]:
# Requisição
try:
    response = requests.get(infectiousdiseases_url)
    response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
except requests.exceptions.RequestException as e:
    print('Erro ao fazer a requisição:', e)
    exit()

infectiousdiseases_html = BeautifulSoup(response.content, 'html.parser')

In [4]:
# Encontrar todos os elementos com a classe "item"
items = infectiousdiseases_html.find_all(class_='item')

# Inicializar uma lista para armazenar as doenças
diseases = []

# Exibir os textos dos elementos <a> dentro dos elementos com a classe "item"
for item in items:
    # Encontrar todos os elementos <a> dentro do elemento com a classe "item"
    a_tags = item.find_all('a')
    for a_tag in a_tags:
        disease = a_tag.text.strip()
        if disease != 'Animal Diseases and Your Health':
            diseases.append(disease)

In [5]:
# Dicionário para armazenar as doenças e sintomas
disease_symptoms = {}

# Itera sobre cada doença na lista de doenças
for disease in diseases:
    # Limpa o nome da doença para criar a URL
    disease_clear = re.sub("[^\w]", "", disease.lower())
    disease_html = f'https://medlineplus.gov/{disease_clear}.html'

    try:
        # Tenta fazer uma requisição HTTP para obter o conteúdo da página da doença
        response = requests.get(disease_html)
        response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
    except requests.exceptions.RequestException as e:
        # Em caso de erro na requisição, exibe uma mensagem de erro e encerra o programa
        print('Erro ao fazer a requisição:', e)
        exit()

    # Analisa o conteúdo HTML da página da doença
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontra o primeiro elemento <h3> que contém "What are the symptoms of"
    symptoms_header = soup.find('h3', string=re.compile(r'What are the symptoms of'))

    if not symptoms_header:
        # Se não houver um cabeçalho de sintomas, adiciona uma lista vazia de sintomas
        symptoms = []
    else:
        # Adiciona uma lista vazia de sintomas
        symptoms = []

        # Encontrar o próximo elemento <h3>
        last_element = symptoms_header.find_next_sibling('h3')

        # Enquanto não chegar ao próximo elemento <h3>
        while symptoms_header != last_element:
            # Avança para o próximo elemento irmão
            symptoms_header = symptoms_header.find_next_sibling()
            #Verifica se existe <li>
            elements = symptoms_header.find_all('li')
            if elements:
                # Adiciona cada item da lista de sintomas ao resultado final
                for element in elements:
                    symptoms.extend([item.strip() for item in element.text.splitlines() if item.strip()])

    # Adiciona a doença e seus sintomas ao dicionário disease_symptoms
    disease_symptoms[disease] = symptoms


In [6]:
# Cria uma lista com as chaves do dicionário disease_symptoms cujos valores são listas vazias
empty_symptoms_keys = [key for key, value in disease_symptoms.items() if not value]

# Imprime o número de chaves com valores vazios
print(len(empty_symptoms_keys))


90


In [7]:
# Dicionário para armazenar as doenças e URL
disease_url = {}

# Itera sobre cada doença na lista de doenças
for disease in empty_symptoms_keys:
    # Limpa o nome da doença para criar a URL
    disease_clear = re.sub("[^\w]", "", disease.lower())
    disease_html = f'https://medlineplus.gov/{disease_clear}.html'

    try:
        # Tenta fazer uma requisição HTTP para obter o conteúdo da página da doença
        response = requests.get(disease_html)
        response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
    except requests.exceptions.RequestException as e:
        # Em caso de erro na requisição, adiciona uma string vazia de sintomas ao dicionário
        print(f'Erro ao fazer a requisição para {disease}: {e}')
        disease_url[disease] = ''
        continue

    # Analisa o conteúdo HTML da página da doença
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar o elemento com o id 'section95'
    section95 = soup.find('div', id='section95')

    if not section95:
        # Se não houver um cabeçalho de sintomas, adiciona uma string vazia de sintomas ao dicionário
        disease_url[disease] = ''
    else:
        # Extrai a URL do href dentro do elemento 'a'
        url = section95.find('a')['href']
        disease_url[disease] = url

In [8]:
# Cria um novo dicionário com as entradas que têm valor ''
disease_url_empty = {key: value for key, value in disease_url.items() if value == ''}

# Obtém o número de entradas no novo dicionário
print(len(disease_url_empty))


68


In [9]:
# Cria um novo dicionário sem as entradas que têm valor ''
disease_url_cleaned = {key: value for key, value in disease_url.items() if value != ''}

# Obtém o número de entradas no novo dicionário
print(len(disease_url_cleaned))

22


In [10]:
# Cria um novo dicionário com as entradas que contêm "cdc" na URL
disease_url_cdc = {key: value for key, value in disease_url_cleaned.items() if 'cdc' in value}

# Exibe o novo dicionário
disease_url_cdc

{'Aspergillosis': 'https://www.cdc.gov/fungal/diseases/aspergillosis/symptoms.html',
 'Anthrax': 'https://www.cdc.gov/anthrax/symptoms/index.html',
 'Tick Bites': 'https://www.cdc.gov/ticks/symptoms.html',
 'Plague': 'https://www.cdc.gov/plague/symptoms/',
 'Chickenpox': 'https://www.cdc.gov/chickenpox/about/symptoms.html',
 'Valley Fever': 'https://www.cdc.gov/fungal/diseases/coccidioidomycosis/symptoms.html',
 'Haemophilus Infections': 'https://www.cdc.gov/hi-disease/about/symptoms.html',
 'Rabies': 'https://www.cdc.gov/rabies/symptoms/',
 'Listeria Infections': 'https://www.cdc.gov/listeria/symptoms.html',
 'Tetanus': 'https://www.cdc.gov/tetanus/about/symptoms-complications.html',
 'Measles': 'https://www.cdc.gov/measles/symptoms/signs-symptoms.html',
 'Mumps': 'https://www.cdc.gov/mumps/about/signs-symptoms.html',
 'Norovirus Infections': 'https://www.cdc.gov/norovirus/about/symptoms.html',
 'Pneumococcal Infections': 'https://www.cdc.gov/pneumococcal/about/symptoms-complications.

In [11]:
# Acessa o valor da URL do primeiro item do dicionário
url = next(iter(disease_url_cleaned.values()))

try:
    # Tenta fazer uma requisição HTTP para obter o conteúdo da página da doença
    response = requests.get(url)
    response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
except requests.exceptions.RequestException as e:
    # Em caso de erro na requisição, adiciona uma string vazia de sintomas ao dicionário
    print(f'Erro ao fazer a requisição para {url}: {e}')


# Analisa o conteúdo HTML da página da doença
soup = BeautifulSoup(response.content, 'html.parser')